# Projeto Integrador 3. Prova de conceito de uma rede Blockchain

### Univesp - Polo Caraguatatuba - Junho 2019

---

### Requisitos prévios

Para executar esse código será necessário instalar:

- [Python 3.6](https://www.python.org/) ou superior (de preferência a distribuição [Anaconda](https://www.anaconda.com/));
- [Flask 0.12.2](http://flask.pocoo.org/) ou superior.

Para testar a API usamos o software [Postman](https://www.getpostman.com/). Mas, devido a sua simplicidade, também poderá ser testada com comandos como `wget`, `curl` ou diretamente no navegador web.

### Código explicado

O primeiro passo será a importação dos módulos que não estão presentes na biblioteca padrão do Python, necessários para a execução do código.

- [datetime](https://docs.python.org/3/library/datetime.html), para manipulação de dados sobre tempo e datas.
- [hashlib](https://docs.python.org/3/library/hashlib.html), para encriptação dos dados pelo algoritmo SHA256.
- [json](https://docs.python.org/3/library/json.html), para manipulação de objetos JSON.

In [1]:
import datetime
import hashlib
import json

Também vamos precisar importar os objetos `Flask` e `jsonify` do microframework para processar requisições HTML:

In [2]:
from flask import Flask, jsonify

A classe `Blockchain` define as operações mais básicas, como:

- Criação de um novo bloco;
- Obter o bloco prévio;
- Prova de trabalho (proof of work);
- Codificação do bloco pelo algoritmo SHA256;
- Validação da cadeia.

O objeto `bloco` é um modelo simplificado de um bloco em uma cadeia blockchain. Ele é representado por uma estrutura do tipo dicionário (key, value) com as seguintes chaves:

- `index`, o índice que representa a posição do bloco na cadeia;
- `timestamp`, com os dados de data e hora fornecidos pelo módulo `datetime`;
- `prova`, o resultado da prova de trabalho realizado durante a mineração;
- `hash_anterior`, o código hash produto da criptografia do bloco precedente.


In [3]:
class Blockchain:

    def __init__(self):
        self.cadeia = []
        self.crear_bloco(prova=1, hash_anterior='0')

    def crear_bloco(self, prova, hash_anterior):
        bloco = {'index': len(self.cadeia) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'prova': prova,
                 'hash_anterior': hash_anterior}
        self.cadeia.append(bloco)
        return bloco

    def obter_bloco_anterior(self):
        return self.cadeia[-1]

    def prova_de_trabalho(self, prova_anterior):
        nova_prova = 1
        checar_prova = False
        while checar_prova is False:
            hashing = hashlib.sha256(str(nova_prova**2 - prova_anterior**2).encode()).hexdigest()
            if hashing[:4] == '0000':
                checar_prova = True
            else:
                nova_prova += 1
        return nova_prova
    
    def hash(self, bloco):
        bloco_codificado = json.dumps(bloco, sort_keys=True).encode()
        return hashlib.sha256(bloco_codificado).hexdigest()
    
    def cadeia_validada(self, cadeia):
        bloco_previo = cadeia[0]
        bloco_index = 1
        while bloco_index < len(cadeia):
            bloco = cadeia[bloco_index]
            if bloco['hash_anterior'] != self.hash(bloco_previo):
                return False
            prova_anterior = bloco_previo['prova']
            prova = bloco['prova']
            hashing = hashlib.sha256(str(prova**2 - prova_anterior**2).encode()).hexdigest()
            if hashing[:4] != '0000':
                return False
            bloco_previo = bloco
            bloco_index += 1
        return True

Aqui criaremos o aplicativo web, instanciando a classe `Flask` do micro framework do mesmo nome, e criamos também uma instância da clase `Blockchain`, a qual já tínhamos declarado previamente:

In [ ]:
app = Flask(__name__)
blockchain = Blockchain()

Na sequência, definimos os end points (API) para o usuário realizar as requisições HTML. Eles são:

- `minerar_bloco`, aqui é feita a mineração para a criação do novo bloco por meio da prova de trabalho;
- `obter_cadeia`, com esse end point obtemos e visualizamos os dados armazenados na cadeia;
- `validar_cadeia`, aqui verificamos se a cadeia não tem sido adulterada.

In [ ]:
@app.route('/minerar_bloco', methods=['GET'])
def minerar_bloco():
    bloco_anterior = blockchain.obter_bloco_anterior()
    prova_anterior = bloco_anterior['prova']
    prova = blockchain.prova_de_trabalho(prova_anterior)
    hash_anterior = blockchain.hash(bloco_anterior)
    bloco = blockchain.crear_bloco(prova, hash_anterior)
    response = {'message': 'Parabéns, você acabou de minerar um bloco!',
                'index': bloco['index'],
                'timestamp': bloco['timestamp'],
                'prova': bloco['prova'],
                'hash_anterior': bloco['hash_anterior']}
    return jsonify(response), 200

@app.route('/obter_cadeia', methods=['GET'])
def obter_cadeia():
    response = {'cadeia': blockchain.cadeia,
                'cumprimento': len(blockchain.cadeia)}
    return jsonify(response), 200

@app.route('/validar_cadeia', methods=['GET'])
def validar_cadeia():
    es_valido = blockchain.cadeia_validada(blockchain.cadeia)
    if es_valido:
        response = {'message': 'Beleza! O Blockchain está validado.'}
    else:
        response = {'message': 'Ops, temos um problema aqui. O Blockchain não foi validado...'}
    return jsonify(response), 200

A continuação executamos o aplicativo no servidor de testes do próprio `Flask`. O servidor receberá as requisições na porta `5000` do `localhost`.

In [ ]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


As requisições podem ser testadas via navegado web, `wget`, `curl` ou `postman` nas seguintes URL:

- [http://localhost:5000/minerar_bloco](http://localhost:5000/minerar_bloco)
- [http://localhost:5000/obter_cadeia](http://localhost:5000/obter_cadeia)
- [http://localhost:5000/validar_cadeia](http://localhost:5000/validar_cadeia)